In [314]:
import thinkplot
import thinkstats2
import pandas as pd
import numpy as np
import scipy.stats as ss
from fractions import Fraction

##Seaborn for fancy plots. 
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams["figure.figsize"] = (15,5)

# Bayes and Updates

We will solve the Monty Hall problem, but first we can work on some more simple examples and build out a method for using Bayes probability calculations. Remember the Bayes theorem from before:

$ P(A|B) = \frac{P(A) P(B|A)}{P(B)} $

Suppose you have two high school classes of 40 students - class A and class B. Each class has some failing students:
<ul>
<li> Class A has 10 failing students, 30 passing ones. 
<li> Class B has 20 failing students, 20 passing ones.
</ul>

<b>If we randomly select one failing student, what is the probability they are from Class A?</b>

We can calculate this out as a table. First, another way to think of Bayes....

## Diachronic Bayes

There is another way to think of Bayes's theorem: it gives us a way to
update the probability of a hypothesis, $H$, given some body of data, $D$.

This interpretation is "diachronic", which means "related to change over time"; in this case, the probability of the hypotheses changes as we see new data.

Rewriting Bayes's theorem with $H$ and $D$ yields:

$ P(H|D) = \frac{P(H)~P(D|H)}{P(D)} $

In this interpretation, each term has a name:

-  $P(H)$ is the probability of the hypothesis before we see the data, called the prior probability, or just **prior**.

-  $P(H|D)$ is the probability of the hypothesis after we see the data, called the **posterior**.

-  $P(D|H)$ is the probability of the data under the hypothesis, called the **likelihood**.

-  $P(D)$ is the **total probability of the data**, under any hypothesis.

Sometimes we can compute the prior based on background information. For example, the classroom problem specifies that we choose a student at random with equal probability.

In other cases the prior is subjective; that is, reasonable people might disagree, either because they use different background information or because they interpret the same information differently.

### Example - Passing and Failing

From above:
<ul>
<li> Class A has 10 failing students, 30 passing ones. 
<li> Class B has 20 failing students, 20 passing ones.
</ul>

First, we can build a table to hose our work. 

In [315]:
table = pd.DataFrame(index=['Class A', 'Class B'])
table

""
Class A
Class B


#### Calculate Prior Probability

We can then add in the first part - the prior probability. 

We can also think of this as "the probability before we know anything else" - here we are just finding the probability of them being in Class A, without adding any other information. 

In [316]:
table['prior'] = 1/2, 1/2
table

,prior
Class A,0.5
Class B,0.5


#### Calculate Likelihoods

Add the likelihoods in...
For class A there is a 25% chance of a student failing.
For class B, it is 50%

We can also think of the likelihoods explicitly as conditional statements:
<ul>
<li> E.g. "If I choose from class B, what is the likelihood of getting a failiure?"
<li> Or, give the prior is true, now what is the probability?
</ul>

You are assuming the "Question part" of the original goal - what are the chances this class provides a failiure, given the stipulation of the prior probability. 

<ul>
<li> If I choose A, 10 out of 40 are failing, so the chances are 1/4
<li> If I choose B, 20 out of 40 are failing, so the chances are 1/2
</ul>

In [317]:

table['likelihood'] = 1/4, 1/2
table

,prior,likelihood
Class A,0.5,0.25
Class B,0.5,0.50


#### Calculate Interim Probabilities

Next, multiply the two probabilities together:
<ul>
<li>E.g. There's a 50% chance of choosing class B, and if I do, there's a 50% chance of getting a fail. 
</ul>
We label this column the unnorm - or unnormalized probabilities. This is because they are both accurate probabilities, but they are not normalized - they do not sum to 1. If we look a little closer, they are also part our boy Bayes' Theorem:
<ul>
<li>The numerator of Bayes is a probability multiplied by a conditional, which is the unnorm value.
<li>The denomenator is the total probability - there are only 2 cases here, one must be true, so it is the sum of the unnorms.
</ul>

We can also think of these probabilities as being "in terms of" the total possible outcomes - i.e. 12.5 of <i>all</i> students are failing in Class A, while 25% of students is failing in B. The other 62.5% are passing, in either class, so they aren't really a consideration in the calculation we are doing. 

In [318]:
#Calculate unnormalized probabilities
table['unnorm'] = table['prior'] * table['likelihood']
table

,prior,likelihood,unnorm
Class A,0.5,0.25,0.125
Class B,0.5,0.50,0.250


As a check, we can demonstrate that last point is true:

- Calculate the total probability of getting a fail by summing the unnorms.

- Calculate the total probability of getting a fail by direct calculation. 

In [319]:
prob_data = table['unnorm'].sum()
print("Unnorms:", prob_data)
probDirect = (10+20)/(40+40) #The overall fail chances - 30 total failiures, 80 total students. 
print("Direct:", probDirect)

Unnorms: 0.375
Direct: 0.375


#### Calculate Posterior Probabilities

Now we can normalize - or make the probs total to 1. We just divide by that total probability. This gives us the posterior probabilities, answering our original question:

$ P(Class A | Failing) $

As well as giving us the other probabilites, for free. 

This step just shifts that probability that we calculated above to be "out of" the total we care about (failing people), rather than the entire total of all students. 

In [320]:
table['posterior'] = table['unnorm'] / prob_data
table

,prior,likelihood,unnorm,posterior
Class A,0.5,0.25,0.125,0.333333
Class B,0.5,0.50,0.250,0.666667


### Table Update Formula

We can wrap those last steps un into a formula, since they are just calculating from the probabilites that we've provided.

In [321]:
def update(table):
    """Compute the posterior probabilities."""
    table['unnorm'] = table['prior'] * table['likelihood']
    prob_data = table['unnorm'].sum()
    table['posterior'] = table['unnorm'] / prob_data
    return prob_data

## Dice problem

Suppose I have a box with a 6-sided die, an 8-sided die, and a 12-sided die. I choose one of the dice at random, roll it, and report that the outcome is a 1. What is the probability that I chose the 6-sided die?

In [322]:
#Create the table
dice = pd.DataFrame(index=["Six Side", "Eight Side", "12 Side"])
dice["prior"] = Fraction(1,3)
dice

,prior
Six Side,1/3
Eight Side,1/3
12 Side,1/3


In [323]:
#The probability for each die being a 1, given I pick it.
dice["likelihood"] = Fraction(1, 6), Fraction(1, 8), Fraction(1, 12)
dice

,prior,likelihood
Six Side,1/3,1/6
Eight Side,1/3,1/8
12 Side,1/3,1/12


In [324]:
#Update to finish
update(dice)
dice

,prior,likelihood,unnorm,posterior
Six Side,1/3,1/6,1/18,4/9
Eight Side,1/3,1/8,1/24,1/3
12 Side,1/3,1/12,1/36,2/9


## The Monty Hall Problem

Next we'll use a Bayes table to solve one of the most contentious problems in probability.

The Monty Hall problem is based on a game show called *Let's Make a Deal*. If you are a contestant on the show, here's how the game works:
<ul>
<li> The host, Monty Hall, shows you three closed doors -- numbered 1, 2, and 3 -- and tells you that there is a prize behind each door.
<li> One prize is valuable (traditionally a car), the other two are less valuable (traditionally goats).
<li> The object of the game is to guess which door has the car. If you guess right, you get to keep the car.
</ul>

The key - after you pick a door, Monty will open another, revealing a goat. Then Monty offers you the option to stick with your original choice or switch to the remaining unopened door. To maximize your chance of winning the car, should you stick with Door 1 or switch to Door 2?

To answer this question, we have to make some assumptions about the behavior of the host (these are parts of the general rules of the game):
<ul>
<li> Monty always opens a door and offers you the option to switch.
<li> He never opens the door you picked or the door with the car.
<li> If you choose the door with the car, he chooses one of the other doors at random.
</ul>

In [325]:
#Start off - initially the chances are equal for each door. 
#So the prior probabilities are all 1/3
monty = pd.DataFrame(index=["Door 1", "Door 2", "Door 3"])
monty["prior"] = Fraction(1,3)
monty

,prior
Door 1,1/3
Door 2,1/3
Door 3,1/3


Now we need to decide what door we want - let's get that whip. 

<b>We'll assume we pick door 1.</b>

Then Monty opens one of the other doors, to us it is random. When he does so it gives us the likelihoods.

<b>We'll assume he opens door 3 - remember he always opens a goat door, not the car</b>

Now, that we know that it isn't door 3 (that's open, it is a goat). Remember, each one is a hypothetical - if we are in this "class" (door choice), what is the probability of "success" (a car there)?: 


We can think about this by carefully defining the problem - What are the odds that Monty opened Door 3, given that the Car is in Door X:
<ul>
<li>The likelihood he'd open Door 3 if the car is there is 0 - we can see the goat, and that's the rules. 
<li>The likelihood he'd open Door 3 if the car is in Door 2 is 1 - he'd be forced to by the rules of the game, you picked Door 1, Door 2 has the car, so he can only open Door 3.
<li>The linkelihood he'd open Door 3 if the car is in Door 1 is 1/2 - he just picks randomly 
</ul>

key - we don't really know the probability the car is in Door X directly. We can use the probability that the Door is opened, and the rules of the game to calculate it. 

In [326]:
#We can update the table
monty["likelihood"] = Fraction(1,2),1,0
monty

,prior,likelihood
Door 1,1/3,1/2
Door 2,1/3,1
Door 3,1/3,0


In [327]:
update(monty)
monty

,prior,likelihood,unnorm,posterior
Door 1,1/3,1/2,1/6,1/3
Door 2,1/3,1,1/3,2/3
Door 3,1/3,0,0,0


Showing it mathmatically requires a bunch of derivation: https://en.wikipedia.org/wiki/Monty_Hall_problem

Alternate explaination that I think is the most clear way to imagine it: Initially there is a 1/3 chance of the car being behind each door. However, after you choose those odds change, due to the rules of the game:
<ul>
<li>The chances it is in your door is still 1/3.
<li>The chances it is not in your door is 2/3.
<li>The door opening part sets the odds for one door to 0, so that 2/3 is contained entirely in one door. 
</ul>

The entire point of this problem is to be unituitive, so having it be confusing is normal. 
<hr>

<h3>Exercise Scenario</h3>

Suppose you are placing a sports bet on your favorite team - The Bayes. You know a few things:
<ul>
<li>The Bayes have a 50% chance of winning a game. (Based on past performance)
<li>The Bayes have had a 10% chance of having rain in their games in Bayes Stadium.
<li>However, in games that The Bayes have won in Bayes Statium, there's be a 11% chance of rain. 
<li>
<li>P(W) = 50%
<li>P(R) = 10%
<li>P(R|W) = 11%
</ul>

<b>What is the probability that The Bayes win if it rains? </b>

In [328]:
#Build Table
bet = pd.DataFrame(index=["Win", "Loss"])
bet["prior"] = Fraction(1,2)
bet 

,prior
Win,1/2
Loss,1/2


In [329]:
#Add in the likelihoods.
#Given that we have a win/loss, how likely is rain. 
bet["likelihood"] = .11, .1
bet

,prior,likelihood
Win,1/2,0.11
Loss,1/2,0.10


In [330]:
#Update to complete. 
update(bet)
bet

,prior,likelihood,unnorm,posterior
Win,1/2,0.11,0.055,0.52381
Loss,1/2,0.10,0.05,0.47619


### Bigger Bayes

What if we want to know the odds that a car is stolen given it is a BMW, black, and at night? 

In [331]:
df_car = pd.read_csv("data/vehicle_stolen_dataset.csv", names=["ID", "Make", "Color", "Time", "Stolen"])
df_car.head(20)

,ID,Make,Color,Time,Stolen
0,N001,BMW,black,night,yes
1,N002,Audi,black,night,no
2,N003,NISSAN,black,night,yes
3,N004,VEGA,red,day,yes
4,N005,BMW,blue,day,no
5,N006,Audi,black,day,yes
6,N007,VEGA,red,night,no
7,N008,Audi,blue,day,yes
8,N009,VEGA,black,day,yes
9,N010,NISSAN,blue,day,no


#### One Step at a Time

Calculate prior probability - will it be stolen? 

In [332]:
car = pd.DataFrame(index=["Stolen", "Not Stolen"])
tmp = df_car[df_car["Stolen"]=="yes"]
tmp2 = df_car[df_car["Stolen"] == "no"]

stolen = len(tmp) / len(df_car)
car["prior"] = stolen, (1 - stolen)
car

,prior
Stolen,0.65
Not Stolen,0.35


#### Update Probability

Add the update of "BMW" as the car model. 

In [333]:
if_stolen = len(tmp[tmp["Make"] == "BMW"]) / len(tmp)
if_notS = len(tmp2[tmp2["Make"] == "BMW"]) / len(tmp2)

In [334]:
car["likelihood"] = if_stolen, if_notS
car

,prior,likelihood
Stolen,0.65,0.307692
Not Stolen,0.35,0.285714


#### Normalize

In [335]:
update(car)
car

,prior,likelihood,unnorm,posterior
Stolen,0.65,0.307692,0.2,0.666667
Not Stolen,0.35,0.285714,0.1,0.333333


#### Intermission

Now we have <i><b>an</b></i> answer, what is the probability that my car will be stolen if it is a BMW. That's not the goal though, we must persist. This is 4/6, which matches what we have if we just look at the table and count. 

In [336]:
#car2 = car[["posterior"]]
#car2 = car2.rename(columns={"posterior": "prior"})
car2 = car[["unnorm"]]
car2 = car2.rename(columns={"unnorm": "prior"})
car2

,prior
Stolen,0.2
Not Stolen,0.1


#### Add New Likelihood

Next up is color. Add the likelihoods for the car being black if it is stolen/not-stolen. 

In [337]:
isblack = len(tmp[tmp["Color"] == "black"]) / len(tmp)
isblack2 = len(tmp2[tmp2["Color"] == "black"]) / len(tmp2)

In [338]:
car2["likelihood"] = isblack, isblack2
car2

,prior,likelihood
Stolen,0.2,0.615385
Not Stolen,0.1,0.142857


#### Update


In [339]:
update(car2)
car2

,prior,likelihood,unnorm,posterior
Stolen,0.2,0.615385,0.123077,0.896
Not Stolen,0.1,0.142857,0.014286,0.104


#### Intermission #2

Still not quite there...

In [340]:
#car3 = car2[["posterior"]]
#car3 = car3.rename(columns={"posterior": "prior"})
car3 = car2[["unnorm"]]
car3 = car3.rename(columns={"unnorm": "prior"})
car3

,prior
Stolen,0.123077
Not Stolen,0.014286


#### Likelihoods

In [341]:
isnight = len(tmp[tmp["Time"] == "night"]) / len(tmp)
isnight2 = len(tmp2[tmp2["Time"] == "night"]) / len(tmp2)

In [342]:
car3["likelihood"] = isnight, isnight2
car3

,prior,likelihood
Stolen,0.123077,0.461538
Not Stolen,0.014286,0.285714


In [343]:
update(car3)
car3

,prior,likelihood,unnorm,posterior
Stolen,0.123077,0.461538,0.056805,0.932963
Not Stolen,0.014286,0.285714,0.004082,0.067037


In [344]:
car3["unnorm"] / .05

Stolen        1.136095
Not Stolen    0.081633
Name: unnorm, dtype: float64

#### Check Result

We think that the probability of my car being stolen <b>if</b> it is a BMW, black, and at night is about 93%. We can check that. 

In [345]:
tmp1 = df_car[df_car["Make"] == "BMW"] # how many are BMW
tmp2 = tmp1[tmp1["Color"] == "black"] # of those, how many are black
tmp3 = tmp2[tmp2["Time"] == "night"] #of those, how many are at night
tmp3

,ID,Make,Color,Time,Stolen
0,N001,BMW,black,night,yes
10,N011,BMW,black,night,yes


In [346]:
p_BMW = len(df_car[df_car["Make"] == "BMW"]) / len(df_car)
p_Black = len(df_car[df_car["Color"] == "black"]) / len(df_car)
p_Night = len(df_car[df_car["Time"] == "night"]) / len(df_car)
den = p_BMW * p_Black * p_Night
p_BMW, p_Black, p_Night, den

(0.3, 0.45, 0.4, 0.054000000000000006)

In [347]:
p_stolen = len(tmp)/len(df)
pBMW_stolen = len(tmp[tmp["Make"] == "BMW"]) / len(tmp)
pBlack_stolen = len(tmp[tmp["Color"] == "black"]) / len(tmp)
pNight_stolen = len(tmp[tmp["Time"] == "night"]) / len(tmp)

numer = (pBMW_stolen * pBlack_stolen * pNight_stolen * p_stolen)


p_stolen, pBMW_stolen, pBlack_stolen, pNight_stolen, numer

(0.65,
 0.3076923076923077,
 0.6153846153846154,
 0.46153846153846156,
 0.05680473372781066)

In [348]:
numer / den

1.0519395134779752

### Multiple Bayes

What is the probability of playing when the weather is sunny, and the temperature is cool.

In [350]:
dfw = pd.read_csv("data/weather.txt", sep="\t")
dfw

,Outlook,Temp,Humidity,Windy,Play
0,Rainy,Hot,High,f,no
1,Rainy,Hot,High,t,no
2,Overcast,Hot,High,f,yes
3,Sunny,Mild,High,f,yes
4,Sunny,Cool,Normal,f,yes
5,Sunny,Cool,Normal,t,no
6,Overcast,Cool,Normal,t,yes
7,Rainy,Mild,High,f,no
8,Rainy,Cool,Normal,f,yes
9,Sunny,Mild,Normal,f,yes


#### Direct Calculation

In [351]:
total = len(dfw)
dfPlay = dfw[dfw["Play"] == "yes"]
dfNoPlay = dfw[dfw["Play"] == "no"]
playTotal = Fraction(len(dfPlay))

#print(len(dfw[dfw["Play"] == "yes"]))

#pPlay = Fraction(len(dfw[dfw["Play"] == "yes"]) / total)
#pOutlook = len(dfw[dfw["Outlook"] == "Sunny"]) / total
#pTemp = len(dfw[dfw["Temp"] == "Cool"]) / total
pPlay = Fraction(len(dfw[dfw["Play"] == "yes"]), total)
pOutlook = Fraction(len(dfw[dfw["Outlook"] == "Sunny"]), total)
pTemp = Fraction(len(dfw[dfw["Temp"] == "Cool"]), total)

print(total, pPlay, pOutlook, pTemp)

14 9/14 5/14 2/7


In [352]:
# Conditionals
pOutlookPlay = len(dfPlay[dfPlay["Outlook"] == "Sunny"]) / playTotal
pTempPlay = len(dfPlay[dfPlay["Temp"] == "Cool"]) / playTotal

print(playTotal, pOutlookPlay, pTempPlay)

9 1/3 1/3


In [353]:
like_play = (pOutlookPlay * pTempPlay * pPlay)
like_play

Fraction(1, 14)

In [354]:
# Total
(pOutlookPlay * pTempPlay * pPlay) / (pOutlook * pTemp)

Fraction(7, 10)

#### Results



### Tables

In [355]:
weather = pd.DataFrame(index=["Play", "Not Play"])

weather["prior"] = pPlay, (1 - pPlay)
weather

,prior
Play,9/14
Not Play,5/14


Add Outlook

In [356]:
if_playOut = Fraction(len(dfPlay[dfPlay["Outlook"] == "Sunny"]), len(dfPlay))
if_notOut = Fraction(len(dfNoPlay[dfNoPlay["Outlook"] == "Sunny"]), len(dfNoPlay))
weather["likelihood"] = if_playOut, if_notOut
weather

,prior,likelihood
Play,9/14,1/3
Not Play,5/14,2/5


In [357]:
update(weather)
weather

,prior,likelihood,unnorm,posterior
Play,9/14,1/3,3/14,3/5
Not Play,5/14,2/5,1/7,2/5


#### Add Temp

In [358]:
#car3 = car2[["posterior"]]
#car3 = car3.rename(columns={"posterior": "prior"})
w2 = weather[["unnorm"]]
w2 = w2.rename(columns={"unnorm": "prior"})
w2

,prior
Play,3/14
Not Play,1/7


In [359]:
if_playTemp = Fraction(len(dfPlay[dfPlay["Temp"] == "Cool"]), len(dfPlay))
if_notTemp = Fraction(len(dfNoPlay[dfNoPlay["Temp"] == "Cool"]), len(dfNoPlay))
w2["likelihood"] = if_playTemp, if_notTemp
w2

,prior,likelihood
Play,3/14,1/3
Not Play,1/7,1/5


In [360]:
update(w2)
w2

,prior,likelihood,unnorm,posterior
Play,3/14,1/3,1/14,5/7
Not Play,1/7,1/5,1/35,2/7


#### Results

We would predict this as a "play", since we are about 71% likely to think that is a play. 

### Sunny, Hot, Normal, False

In [361]:
weatherBig = pd.DataFrame(index=["Play", "Not Play"])
weatherBig["prior"] = pPlay, (1 - pPlay)
weatherBig

,prior
Play,9/14
Not Play,5/14


##### Outlook

In [362]:
if_playOut2 = Fraction(len(dfPlay[dfPlay["Outlook"] == "Sunny"]), len(dfPlay))
if_notOut2 = Fraction(len(dfNoPlay[dfNoPlay["Outlook"] == "Sunny"]), len(dfNoPlay))
print(Fraction(if_playOut2), Fraction(if_notOut2))
weatherBig["likelihood"] = if_playOut2, if_notOut2
update(weatherBig)
weatherBig

1/3 2/5


,prior,likelihood,unnorm,posterior
Play,9/14,1/3,3/14,3/5
Not Play,5/14,2/5,1/7,2/5


##### Temperature

In [363]:
wB1 = weatherBig[["unnorm"]]
wB1 = wB1.rename(columns={"unnorm": "prior"})
wB1

,prior
Play,3/14
Not Play,1/7


In [364]:
if_playTemp2 = Fraction(len(dfPlay[dfPlay["Temp"] == "Hot"]), len(dfPlay))
if_notTemp2 = Fraction(len(dfNoPlay[dfNoPlay["Temp"] == "Hot"]), len(dfNoPlay))
print(Fraction(if_playTemp2), Fraction(if_notTemp2))
wB1["likelihood"] = if_playTemp2, if_notTemp2
update(wB1)
wB1

2/9 2/5


,prior,likelihood,unnorm,posterior
Play,3/14,2/9,1/21,5/11
Not Play,1/7,2/5,2/35,6/11


##### Humidity

In [365]:
wB2 = wB1[["unnorm"]]
wB2 = wB2.rename(columns={"unnorm": "prior"})
wB2

,prior
Play,1/21
Not Play,2/35


In [366]:
if_playHum2 = Fraction(len(dfPlay[dfPlay["Humidity"] == "Normal"]) , len(dfPlay))
if_notHum2 = Fraction(len(dfNoPlay[dfNoPlay["Humidity"] == "Normal"]) , len(dfNoPlay))
print(Fraction(if_playHum2), Fraction(if_notHum2))
wB2["likelihood"] = if_playHum2, if_notHum2
update(wB2)
wB2

2/3 1/5


,prior,likelihood,unnorm,posterior
Play,1/21,2/3,2/63,25/34
Not Play,2/35,1/5,2/175,9/34


##### Wind

In [367]:
wB3 = wB2[["unnorm"]]
wB3 = wB3.rename(columns={"unnorm": "prior"})
wB3

,prior
Play,2/63
Not Play,2/175


In [368]:
if_playWind2 = Fraction(len(dfPlay[dfPlay["Windy"] == "f"]) , len(dfPlay))
if_notWind2 = Fraction(len(dfNoPlay[dfNoPlay["Windy"] == "f"]) , len(dfNoPlay))
print(Fraction(if_playWind2), Fraction(if_notWind2))
wB3["likelihood"] = if_playWind2, if_notWind2
update(wB3)
wB3

2/3 2/5


,prior,likelihood,unnorm,posterior
Play,2/63,2/3,4/189,125/152
Not Play,2/175,2/5,4/875,27/152
